In [21]:
from datetime import datetime
import time
import sqlite3
import main
import json

# milliseconds to datetime
def milliseconds_to_datetime(milliseconds):
    return datetime.fromtimestamp(milliseconds / 1000.0)

# get the friends activity json object from the main.py
friends_activity_json = main.get_friends_activity_json()

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Create a SQLite database file.
conn = sqlite3.connect('friends_activity.sqlite')

# Connect to the SQLite database.
cur = conn.cursor()

# Check if the table exists.
cur.execute('''
    CREATE TABLE IF NOT EXISTS users (
        user_uri TEXT PRIMARY KEY,
        user_name TEXT NOT NULL,
        user_imageUrl TEXT NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS tracks (
        track_uri TEXT PRIMARY KEY,
        track_name TEXT NOT NULL,
        track_imageUrl TEXT NOT NULL,
        track_album_uri TEXT NOT NULL,
        track_album_name TEXT NOT NULL,
        track_artist_uri TEXT NOT NULL,
        track_artist_name TEXT NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS contexts (
        context_uri TEXT PRIMARY KEY,
        context_name TEXT NOT NULL
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS listenings (
        user_id TEXT NOT NULL,
        track_id TEXT NOT NULL,
        context_id TEXT NOT NULL,
        timestamp INTEGER NOT NULL,
        track_listens_count INTEGER NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users (user_uri),
        FOREIGN KEY (track_id) REFERENCES tracks (track_uri),
        FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
    )
''')

# add friends activity to the database
for friend in friends_activity_json['friends']:
    # add user to the database
    cur.execute("""
        INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
        VALUES (?, ?, ?)
    """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
    # add track to the database
    cur.execute("""
        INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
    # add context to the database
    cur.execute("""
        INSERT OR IGNORE INTO contexts (context_uri, context_name)
        VALUES (?, ?)
    """, (friend['track']['context']['uri'], friend['track']['context']['name']))
    # add listening to the database
    cur.execute("""
        INSERT OR IGNORE INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
        VALUES (?, ?, ?, ?, ?)
    """, (friend['user']['uri'], friend['track']['uri'], friend['track']['context']['uri'], friend['timestamp'], old_count))

# get all the users from the database
cur.execute("""
    SELECT * FROM users
""")
users = cur.fetchall()

for i in users:
    print(i)


# Commit the changes to the database.
conn.commit()

# Close the database connection.
conn.close()


OperationalError: near "OR": syntax error

In [1]:
from datetime import datetime
import time
import sqlite3
import main
import json

# milliseconds to datetime
def milliseconds_to_datetime(milliseconds):
    return datetime.fromtimestamp(milliseconds / 1000.0)

# get the friends activity json object from the main.py
friends_activity_json = main.get_friends_activity_json()

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Use a context manager to connect to the SQLite database.
with sqlite3.connect('friends_activity.sqlite') as conn:
    # Connect to the SQLite database.
    cur = conn.cursor()
    # create the users table with user_id as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY,
            user_uri TEXT NOT NULL,
            user_name TEXT NOT NULL,
            user_imageUrl TEXT NOT NULL
        )
    ''')

    # create the tracks table with track_id as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS tracks (
            track_id INTEGER PRIMARY KEY,
            track_uri TEXT NOT NULL,
            track_name TEXT NOT NULL,
            track_imageUrl TEXT NOT NULL,
            track_album_uri TEXT NOT NULL,
            track_album_name TEXT NOT NULL,
            track_artist_uri TEXT NOT NULL,
            track_artist_name TEXT NOT NULL
        )
    ''')

    # create the contexts table with context_uri as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS contexts (
            context_uri TEXT PRIMARY KEY,
            context_name TEXT NOT NULL
        )
    ''')

    # create the listenings table with user_id and track_id as the foreign keys
    cur.execute('''
        CREATE TABLE IF NOT EXISTS listenings (
            user_id INTEGER NOT NULL,
            track_id INTEGER NOT NULL,
            context_id TEXT NOT NULL,
            timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
            track_listens_count INTEGER NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users (user_id),
            FOREIGN KEY (track_id) REFERENCES tracks (track_id),
            FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
        )
    ''')

    # add friends activity to the database
    for friend in friends_activity_json['friends']:
        # add user to the database
        cur.execute("""
            INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
            VALUES (?, ?, ?)
        """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
        # get the user_id of the inserted user
        cur.execute("""
            SELECT user_id FROM users WHERE user_uri = ?
        """, (friend['user']['uri'],))
        user_id = cur.fetchone()[0]
        # add track to the database
        cur.execute("""
            INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
        # get the track_id of the inserted track
        cur.execute("""
            SELECT track_id FROM tracks WHERE track_uri = ?
        """, (friend['track']['uri'],))
        track_id = cur.fetchone()[0]
        # add context to the database
        cur.execute("""
            INSERT OR IGNORE INTO contexts (context_uri, context_name)
            VALUES (?, ?)
        """, (friend['track']['context']['uri'], friend['track']['context']['name']))
        # check if the user has listened to the same track before
        cur.execute("""
            SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
        """, (user_id, track_id))
        
        # previous_timestamps = cur.fetchall()
        
    users = cur.fetchall()

    for i in users:
        print(i)

# No need to explicitly commit or close the connection as it is handled by the context manager.


In [2]:

# Create a SQLite database file.
conn = sqlite3.connect('friends_activity.sqlite')

# Connect to the SQLite database.
cur = conn.cursor()

# print all the users from the database
cur.execute("""
    SELECT * FROM users
""")
users = cur.fetchall()

print("users: \n user_uri, user_name, user_imageUrl")
for i in users:
    print(i)

# print all the tracks from the database
cur.execute("""
    SELECT * FROM tracks
""")
tracks = cur.fetchall()

print("tracks: \n track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name")
for i in tracks:
    print(i)

# print all the contexts from the database
cur.execute("""
    SELECT * FROM contexts
""")
contexts = cur.fetchall()

print("contexts: \n context_uri, context_name")
for i in contexts:
    print(i)

# print all the listenings from the database
cur.execute("""
    SELECT * FROM listenings
""")
listenings = cur.fetchall()

print("listenings: \n user_id, track_id, context_id, timestamp, track_listens_count")
for i in listenings:
    print(i)


# Commit the changes to the database.
conn.commit()

# Close the database connection.
conn.close()


users: 
 user_uri, user_name, user_imageUrl
(1, 'spotify:user:xbqlbum9p8gknixdovte1d15c', '43giyotin', 'https://i.scdn.co/image/ab67757000003b82850b9f39868e881264d4ba7d')
(2, 'spotify:user:nqm0cupkbep7amqd07yrz0xm1', 'aria', 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2')
(3, 'spotify:user:e3ej04dfm6i46el6bcko7gdyj', 'flutestorm', 'https://i.scdn.co/image/ab67757000003b822ee420ed59b80518f3ba7fbe')
(4, 'spotify:user:13v0l2zagkat9thdoqngm8ki8', 'maja', 'https://i.scdn.co/image/ab67757000003b822630b0839d17ab28f6be5b99')
(5, 'spotify:user:homestar99', 'Eliott', 'https://i.scdn.co/image/ab67757000003b8233e21b088263fa0f21ac5f2b')
(6, 'spotify:user:topsify', 'Topsify', 'https://i.scdn.co/image/ab67757000003b82ad2b502872293bac301b8fd6')
(7, 'spotify:user:compton182', 'ARTV', 'https://i.scdn.co/image/ab67757000003b82c37f6298191d13be6ed2d158')
(8, 'spotify:user:gntab9tp1cc5qipthodlvvsm3', 'suchita', 'https://i.scdn.co/image/ab67757000003b829dc4867ae0babe8b2d882f5a')
(9, 'spot

In [ ]:
from datetime import datetime
import time
import sqlite3
import main
import json

# milliseconds to datetime
def milliseconds_to_datetime(milliseconds):
    return datetime.fromtimestamp(milliseconds / 1000.0)

# get the friends activity json object from the main.py
friends_activity_json = main.get_friends_activity_json()

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Use a context manager to connect to the SQLite database.
with sqlite3.connect('friends_activity.sqlite') as conn:
    # Connect to the SQLite database.
    cur = conn.cursor()
    # create the users table with user_id as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY,
            user_uri TEXT NOT NULL,
            user_name TEXT NOT NULL,
            user_imageUrl TEXT NOT NULL
        )
    ''')

    # create the tracks table with track_id as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS tracks (
            track_id INTEGER PRIMARY KEY,
            track_uri TEXT NOT NULL,
            track_name TEXT NOT NULL,
            track_imageUrl TEXT NOT NULL,
            track_album_uri TEXT NOT NULL,
            track_album_name TEXT NOT NULL,
            track_artist_uri TEXT NOT NULL,
            track_artist_name TEXT NOT NULL
        )
    ''')

    # create the contexts table with context_uri as the primary key
    cur.execute('''
        CREATE TABLE IF NOT EXISTS contexts (
            context_uri TEXT PRIMARY KEY,
            context_name TEXT NOT NULL
        )
    ''')

    # create the listenings table with user_id and track_id as the foreign keys
    cur.execute('''
        CREATE TABLE IF NOT EXISTS listenings (
            user_id INTEGER NOT NULL,
            track_id INTEGER NOT NULL,
            context_id TEXT NOT NULL,
            timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
            track_listens_count INTEGER NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users (user_id),
            FOREIGN KEY (track_id) REFERENCES tracks (track_id),
            FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
        )
    ''')

    # add friends activity to the database
    for friend in friends_activity_json['friends']:
        # add user to the database
        cur.execute("""
            INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
            VALUES (?, ?, ?)
        """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
        # get the user_id of the inserted user
        cur.execute("""
            SELECT user_id FROM users WHERE user_uri = ?
        """, (friend['user']['uri'],))
        user_id = cur.fetchone()[0]
        # add track to the database
        cur.execute("""
            INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
        # get the track_id of the inserted track
        cur.execute("""
            SELECT track_id FROM tracks WHERE track_uri = ?
        """, (friend['track']['uri'],))
        track_id = cur.fetchone()[0]
        # add context to the database
        cur.execute("""
            INSERT OR IGNORE INTO contexts (context_uri, context_name)
            VALUES (?, ?)
        """, (friend['track']['context']['uri'], friend['track']['context']['name']))
        # check if the user has listened to the same track before
        cur.execute("""
            SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
        """, (user_id, track_id))
        
        # previous_timestamps = cur.fetchall()
        
    users = cur.fetchall()

    for i in users:
        print(i)

# No need to explicitly commit or close the connection as it is handled by the context manager.


In [23]:
from datetime import datetime
import time
import sqlite3
import main
import json

# milliseconds to datetime
def milliseconds_to_datetime(milliseconds):
    return datetime.fromtimestamp(milliseconds / 1000.0)

# get the friends activity json object from the main.py
friends_activity_json = main.get_friends_activity_json()

current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# Use a context manager to connect to the SQLite database.
with sqlite3.connect('friends_activity.sqlite') as conn:
    # Connect to the SQLite database.
    cur = conn.cursor()

    # Check if the table exists.
    cur.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_uri TEXT PRIMARY KEY,
            user_name TEXT NOT NULL,
            user_imageUrl TEXT NOT NULL
        )
    ''')

    cur.execute('''
        CREATE TABLE IF NOT EXISTS tracks (
            track_uri TEXT PRIMARY KEY,
            track_name TEXT NOT NULL,
            track_imageUrl TEXT NOT NULL,
            track_album_uri TEXT NOT NULL,
            track_album_name TEXT NOT NULL,
            track_artist_uri TEXT NOT NULL,
            track_artist_name TEXT NOT NULL
        )
    ''')

    cur.execute('''
        CREATE TABLE IF NOT EXISTS contexts (
            context_uri TEXT PRIMARY KEY,
            context_name TEXT NOT NULL
        )
    ''')

    # Add a column for timestamps to the listenings table
    cur.execute('''
        CREATE TABLE IF NOT EXISTS listenings (
            user_id TEXT NOT NULL,
            track_id TEXT NOT NULL,
            context_id TEXT NOT NULL,
            timestamp INTEGER NOT NULL, -- store timestamps as UNIX epoch values
            track_listens_count INTEGER NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users (user_uri),
            FOREIGN KEY (track_id) REFERENCES tracks (track_uri),
            FOREIGN KEY (context_id) REFERENCES contexts (context_uri)
        )
    ''')

    # add friends activity to the database
    for friend in friends_activity_json['friends']:
        # add user to the database
        cur.execute("""
            INSERT OR IGNORE INTO users (user_uri, user_name, user_imageUrl)
            VALUES (?, ?, ?)
        """, (friend['user']['uri'], friend['user']['name'], friend['user']['imageUrl']))
        # add track to the database
        cur.execute("""
            INSERT OR IGNORE INTO tracks (track_uri, track_name, track_imageUrl, track_album_uri, track_album_name, track_artist_uri, track_artist_name)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (friend['track']['uri'], friend['track']['name'], friend['track']['imageUrl'], friend['track']['album']['uri'], friend['track']['album']['name'], friend['track']['artist']['uri'], friend['track']['artist']['name']))
        # add context to the database
        cur.execute("""
            INSERT OR IGNORE INTO contexts (context_uri, context_name)
            VALUES (?, ?)
        """, (friend['track']['context']['uri'], friend['track']['context']['name']))
        
        # check if the user has listened to the same track before
        cur.execute("""
            SELECT timestamp FROM listenings WHERE user_id = ? AND track_id = ?
        """, (friend['user']['uri'], friend['track']['uri']))
        
        previous_timestamps = cur.fetchall()
        
        # if yes, update the timestamp column with the new value and increment the track_listens_count column by 1
        if previous_timestamps:
            cur.execute("""
                UPDATE listenings SET timestamp = ?, track_listens_count = track_listens_count + 1 WHERE user_id = ? AND track_id = ?
            """, (friend['timestamp'], friend['user']['uri'], friend['track']['uri']))
        
        # if no, insert a new row with the user_id, track_id, context_id, timestamp and track_listens_count columns
        else:
            cur.execute("""
                INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
                VALUES (?, ?, ?, ?, ?)
            """, (friend['user']['uri'], friend['track']['uri'], friend['track']['context']['uri'], friend['timestamp'], 1))

    # get all the users from the database
    cur.execute("""
        SELECT * FROM users
    """)
    users = cur.fetchall()

    for i in users:
        print(i)

# No need to explicitly commit or close the connection as it is handled by the context manager.


('spotify:user:sonemic.com', 'sonemic.com', 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f')
('spotify:user:homestar99', 'Eliott', 'https://i.scdn.co/image/ab67757000003b8233e21b088263fa0f21ac5f2b')
('spotify:user:jaime33', 'Irles Music', 'https://i.scdn.co/image/ab67757000003b82cdbf4610bef6efb87adb4daf')
('spotify:user:miles_lamborgenie', 'Miles Lambert-Gilliam', 'https://i.scdn.co/image/ab67757000003b82656acdcdafa5afe6146ee695')
('spotify:user:02aj59ugjm84jmkqaimbjl2hb', 'gaurisha', 'https://i.scdn.co/image/ab67757000003b82bf61348dee9bcfd870474dab')
('spotify:user:xbqlbum9p8gknixdovte1d15c', '43giyotin', 'https://i.scdn.co/image/ab67757000003b82850b9f39868e881264d4ba7d')
('spotify:user:nqm0cupkbep7amqd07yrz0xm1', 'aria', 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2')
('spotify:user:compton182', 'ARTV', 'https://i.scdn.co/image/ab67757000003b82c37f6298191d13be6ed2d158')
('spotify:user:earm28v0wpfycco58h1e64qi4', 'HuzgOnline', 'https://i.scdn.co/

(1, 'spotify:user:sonemic.com', 'sonemic.com', 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f')
(2, 'spotify:user:nqm0cupkbep7amqd07yrz0xm1', 'aria', 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2')
(3, 'spotify:user:sonemic.com', 'sonemic.com', 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f')
(4, 'spotify:user:nqm0cupkbep7amqd07yrz0xm1', 'aria', 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2')


In [25]:
print(len("The blueprint is on a large, brownish yellow sheet of paper. The paper is slightly wrinkled and has a few folds in it. The drawing of the spaceship is in blue ink. The spaceship is a large, sleek, metallic craft. It is shaped like a teardrop, with a long, pointed nose and a wide, flat tail. The ship is covered in strange symbols and markings, which are written in an alien language that is unknown to humans. The text on the blueprint is also in blue ink."))

457


In [25]:
friends_activity_json

{'friends': [{'timestamp': 1682175987406,
   'user': {'uri': 'spotify:user:sonemic.com',
    'name': 'sonemic.com',
    'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'},
   'track': {'uri': 'spotify:track:09hctAvAEUpm7MKl1RBK2j',
    'name': 'Revolve',
    'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27351156e6871791e38b0e29b46',
    'album': {'uri': 'spotify:album:1Ft1QQbUOCgSqumZKYuWhw',
     'name': 'Stoner Witch'},
    'artist': {'uri': 'spotify:artist:6aVjo0xHSiuW5hkasoYSR3',
     'name': 'Melvins'},
    'context': {'uri': 'spotify:playlist:1pFS1lpklMj10fvsOziZ6S',
     'name': 'Front Page Features | Sonemic Selects',
     'index': 0}}},
  {'timestamp': 1682300630805,
   'user': {'uri': 'spotify:user:nqm0cupkbep7amqd07yrz0xm1',
    'name': 'aria',
    'imageUrl': 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2'},
   'track': {'uri': 'spotify:track:2CwP6A4P9tsPH3rCDKgq2D',
    'name': 'Stolen Dog',
    'imageUrl': 'http://i.scdn.

In [23]:

# Create a SQLite database file.
conn = sqlite3.connect('friends_activity.sqlite')

# Connect to the SQLite database.
cur = conn.cursor()


# select all data from listenings table
# cur.execute("SELECT * FROM listenings")

# adding my_dict to listenings table
cur.execute("""
    INSERT INTO listenings (user_id, track_id, context_id, timestamp, track_listens_count)
    VALUES (?, ?, ?, ?, ?)
""", (my_dict['user']['uri'], my_dict['track']['uri'], my_dict['context']['uri'], my_dict['timestamp'], 1))

# commit the changes
conn.commit()

# close the connection
conn.close()


# If the result is None, create the table.

# # create users table if not exists
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS users (
#         user_id INTEGER PRIMARY KEY,
#         user_uri TEXT,
#         user_name TEXT,
#         user_imageUrl TEXT
#     )
# """)

# # create tracks table if not exists
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS tracks (
#         track_id INTEGER PRIMARY KEY,
#         track_uri TEXT,
#         track_name TEXT,
#         track_imageUrl TEXT,
#         track_album_uri TEXT,
#         track_album_name TEXT,
#         track_artist_uri TEXT,
#         track_artist_name TEXT
#     )
# """)

# # create contexts table if not exists
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS contexts (
#         context_id INTEGER PRIMARY KEY,
#         context_uri TEXT,
#         context_name TEXT
#     )
# """)

# # create listenings table if not exists
# cur.execute("""
#     CREATE TABLE IF NOT EXISTS listenings (
#         listening_id INTEGER PRIMARY KEY,
#         user_id INTEGER,
#         track_id INTEGER,
#         context_id INTEGER,
#         timestamp INTEGER,
#         track_listens_count INTEGER,
#         FOREIGN KEY (user_id) REFERENCES users (user_id),
#         FOREIGN KEY (track_id) REFERENCES tracks (track_id),
#         FOREIGN KEY (context_id) REFERENCES contexts (context_id)
#     )
# """)


KeyError: 'context'

In [16]:
my_dict = {'timestamp': 1682175987406,
   'user': {'uri': 'spotify:user:sonemic.com',
    'name': 'sonemic.com',
    'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'},
   'track': {'uri': 'spotify:track:09hctAvAEUpm7MKl1RBK2j',
    'name': 'Revolve',
    'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27351156e6871791e38b0e29b46',
    'album': {'uri': 'spotify:album:1Ft1QQbUOCgSqumZKYuWhw',
     'name': 'Stoner Witch'},
    'artist': {'uri': 'spotify:artist:6aVjo0xHSiuW5hkasoYSR3',
     'name': 'Melvins'},
    'context': {'uri': 'spotify:playlist:1pFS1lpklMj10fvsOziZ6S',
     'name': 'Front Page Features | Sonemic Selects',
     'index': 0}}}

In [4]:

# create a cursor object
cur = conn.cursor()

# select all data from users table
cur.execute("SELECT * FROM listenings")

# fetch all rows as a list of tuples
rows = cur.fetchall()
print(rows)

# loop through each row and print it
for row in rows:
    print(row)

# # close the cursor and connection objects
# cur.close()
# conn.close()

[]


In [30]:

# # Check if the table exists.
# cur.execute('''
#         SELECT name
#         FROM sqlite_master
#         WHERE type = 'table'
#         AND name = 'friends_activity'
#         ''')

# # Store the result of fetchone() in a variable.
# result = cur.fetchone()

# # If the result is None, create the table.
# if result is None:
#   print('Creating table...')
#   cur.execute('''
#         CREATE TABLE friends_activity (
#           timestamp INTEGER,
#           user_uri TEXT,
#           user_name TEXT,
#           user_imageUrl TEXT,
#           track_uri TEXT,
#           track_name TEXT,
#           track_imageUrl TEXT,
#           track_album_uri TEXT,
#           track_album_name TEXT,
#           track_artist_uri TEXT,
#           track_artist_name TEXT,
#           track_context_uri TEXT,
#           track_context_name TEXT,
#           track_context_index INTEGER,
#           track_listens_count INTEGER
#         )
#         ''')

# current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# # fetch result from the database
# # result = cur.execute('''SELECT * FROM friends_activity''')
# result = cur.fetchall()


# # print the result
# for row in result:
#     print(row)


# # Insert the data into the table.
# for friend in friends_activity_json['friends']:
#   cur.execute(''' INSERT INTO friends_activity (timestamp,
#                                 user_uri,
#                                 user_name,
#                                 track_uri,
#                                 track_name,
#                                 track_imageUrl,
#                                 track_album_uri,
#                                 track_album_name,
#                                 track_artist_uri,
#                                 track_artist_name,
#                                 track_context_uri,
#                                 track_context_name,
#                                 track_listens)
#   VALUES (?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?,  ?, ?)''',
#        (friend['timestamp'],
#         friend['user']['uri'],
#         friend['user']['name'],
#         friend['user']['imageUrl'],
#         friend['track']['uri'],
#         friend['track']['name'],
#         friend['track']['imageUrl'],
#         friend['track']['album']['uri'],
#         friend['track']['album']['name'],
#         friend['track']['artist']['uri'],
#         friend['track']['artist']['name'],
#         friend['track']['context']['uri'],
#         friend['track']['context']['name'],
#         # friend['track']['context']['index'],
#         ))


# Close the connection to the SQLite database.
# conn.close()

In [23]:
cur.execute('''SELECT * FROM friends_activity''')
rows = cur.fetchall()
# print(rows)
# print(rows)
for i in rows:
    print(i)

(1682175987406, 'spotify:user:sonemic.com', 'sonemic.com', 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f', 'spotify:track:09hctAvAEUpm7MKl1RBK2j', 'Revolve', 'http://i.scdn.co/image/ab67616d0000b27351156e6871791e38b0e29b46', 'spotify:album:1Ft1QQbUOCgSqumZKYuWhw', 'Stoner Witch', 'spotify:artist:6aVjo0xHSiuW5hkasoYSR3', 'Melvins', 'spotify:playlist:1pFS1lpklMj10fvsOziZ6S', 'Front Page Features | Sonemic Selects', 0)
(1682300630805, 'spotify:user:nqm0cupkbep7amqd07yrz0xm1', 'aria', 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2', 'spotify:track:2CwP6A4P9tsPH3rCDKgq2D', 'Stolen Dog', 'http://i.scdn.co/image/ab67616d0000b273d7049157fae42e2514c1875e', 'spotify:album:1UTUaGgazW903gGukLzLe5', 'Street Halo', 'spotify:artist:0uCCBpmg6MrPb1KY2msceF', 'Burial', 'spotify:album:1UTUaGgazW903gGukLzLe5', 'Street Halo', 0)
(1682399770375, 'spotify:user:02aj59ugjm84jmkqaimbjl2hb', 'gaurisha', 'https://i.scdn.co/image/ab67757000003b82bf61348dee9bcfd870474dab', 'sp

In [19]:
rows = cur.fetchall()
print(rows)

[]


In [20]:
print(rows)
for i in rows:
    print(i)

[]


In [6]:
friends_activity_json

{'friends': [{'timestamp': 1682175987406,
   'user': {'uri': 'spotify:user:sonemic.com',
    'name': 'sonemic.com',
    'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'},
   'track': {'uri': 'spotify:track:09hctAvAEUpm7MKl1RBK2j',
    'name': 'Revolve',
    'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27351156e6871791e38b0e29b46',
    'album': {'uri': 'spotify:album:1Ft1QQbUOCgSqumZKYuWhw',
     'name': 'Stoner Witch'},
    'artist': {'uri': 'spotify:artist:6aVjo0xHSiuW5hkasoYSR3',
     'name': 'Melvins'},
    'context': {'uri': 'spotify:playlist:1pFS1lpklMj10fvsOziZ6S',
     'name': 'Front Page Features | Sonemic Selects',
     'index': 0}}},
  {'timestamp': 1682300630805,
   'user': {'uri': 'spotify:user:nqm0cupkbep7amqd07yrz0xm1',
    'name': 'aria',
    'imageUrl': 'https://i.scdn.co/image/ab67757000003b827948f3a08da74b5adff251e2'},
   'track': {'uri': 'spotify:track:2CwP6A4P9tsPH3rCDKgq2D',
    'name': 'Stolen Dog',
    'imageUrl': 'http://i.scdn.

In [7]:
data = (friend['timestamp'],
        friend['user']['uri'],
        friend['user']['name'],
        friend['user']['imageUrl'],
        friend['track']['uri'],
        friend['track']['name'],
        friend['track']['imageUrl'],
        friend['track']['album']['uri'],
        friend['track']['album']['name'],
        friend['track']['artist']['uri'],
        friend['track']['artist']['name'],
        # friend['track']['context']['uri'],
        # friend['track']['context']['name'],
        # friend['track']['context']['index']
        )
for i in data:
  print(i)

1682175987406
spotify:user:sonemic.com
sonemic.com
https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f
spotify:track:09hctAvAEUpm7MKl1RBK2j
Revolve
http://i.scdn.co/image/ab67616d0000b27351156e6871791e38b0e29b46
spotify:album:1Ft1QQbUOCgSqumZKYuWhw
Stoner Witch
spotify:artist:6aVjo0xHSiuW5hkasoYSR3
Melvins


In [69]:
friends_activity_json

{'friends': [{'timestamp': 1682079384511,
   'user': {'uri': 'spotify:user:xbqlbum9p8gknixdovte1d15c',
    'name': '43giyotin',
    'imageUrl': 'https://i.scdn.co/image/ab67757000003b82850b9f39868e881264d4ba7d'},
   'track': {'uri': 'spotify:track:61KTWO81lZhsYdp244OZAV',
    'name': 'BAROQUE',
    'imageUrl': 'http://i.scdn.co/image/ab67616d0000b273c1471b056b29a6610d87f68e',
    'album': {'uri': 'spotify:album:1wUG4FsRMYhGpinoV3GHls',
     'name': 'BAROQUE'},
    'artist': {'uri': 'spotify:artist:3xMSStWH6Wz2KAJqkZwXLu',
     'name': 'Yabujin'},
    'context': {'uri': 'spotify:album:1wUG4FsRMYhGpinoV3GHls',
     'name': 'BAROQUE',
     'index': 0}}},
  {'timestamp': 1682175987406,
   'user': {'uri': 'spotify:user:sonemic.com',
    'name': 'sonemic.com',
    'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'},
   'track': {'uri': 'spotify:track:09hctAvAEUpm7MKl1RBK2j',
    'name': 'Revolve',
    'imageUrl': 'http://i.scdn.co/image/ab67616d0000b27351156e68717